In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Data_rec_sys/data.zip -d /content/

Archive:  /content/drive/MyDrive/Data_rec_sys/data.zip
   creating: /content/data/
   creating: /content/data/.ipynb_checkpoints/
  inflating: /content/data/aisles.csv  
  inflating: /content/data/chunk_0.pkl  
  inflating: /content/data/departments.csv  
  inflating: /content/data/orders.csv  
  inflating: /content/data/orders_comsum.pkl  
  inflating: /content/data/order_products__prior.csv  
  inflating: /content/data/order_products__train.csv  
  inflating: /content/data/previous_products.pkl  
  inflating: /content/data/prod2vec.pkl  
  inflating: /content/data/products.csv  
  inflating: /content/data/product_embeddings_final.pkl  
  inflating: /content/data/product_period.pkl  
  inflating: /content/data/product_periods_stat.pkl  
  inflating: /content/data/user_aisle_products.pkl  
  inflating: /content/data/user_department_products.pkl  
   creating: /content/data/__MACOSX/
  inflating: /content/data/__MACOSX/._aisles.csv  
  inflating: /content/data/__MACOSX/._departments.csv

# CREATE PRODUCTS

In [ ]:
import pandas as pd
df = pd.read_csv('/content/data/products.csv')
df.head(10)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
5,6,Dry Nose Oil,11,11
6,7,Pure Coconut Water With Orange,98,7
7,8,Cut Russet Potatoes Steam N' Mash,116,1
8,9,Light Strawberry Blueberry Yogurt,120,16
9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7


In [ ]:
import gc
import pandas as pd
import numpy as np
import os

if __name__ == '__main__':
    path = '/content/data'

    order_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                                                                      'product_id': np.uint16,
                                                                                      'add_to_cart_order':np.uint8,
                                                                                      'reordered': bool})
    orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id':np.uint32,
                                                                  'user_id': np.uint32,
                                                                  'eval_set': 'category',
                                                                  'order_number':np.uint8,
                                                                  'order_dow': np.uint8,
                                                                  'order_hour_of_day': np.uint8
                                                                  })

    print('loaded')

    orders = orders.loc[orders.eval_set == 'prior', :]
    orders_user = orders[['order_id', 'user_id']]
    labels = pd.merge(order_prior, orders_user, on='order_id')
    labels = labels.loc[:, ['user_id', 'product_id']].drop_duplicates()

    print(labels)

    print('save')
    print(labels.shape)
    print(labels.columns)


loaded
          user_id  product_id
0          202279       33120
1          202279       28985
2          202279        9327
3          202279       45918
4          202279       30035
...           ...         ...
32434480    25247       45309
32434481    25247       21162
32434483    25247       35211
32434485    25247       11352
32434486    25247        4600

[13307953 rows x 2 columns]
save
(13307953, 2)
Index(['user_id', 'product_id'], dtype='object')


In [ ]:
orders = orders.loc[orders.eval_set == 'prior', :]
orders_user = orders[['order_id', 'user_id']]
labels = pd.merge(order_prior, orders_user, on='order_id')
labels = labels.loc[:, ['user_id', 'product_id']].drop_duplicates()

In [ ]:
labels.to_pickle('/content/data/previous_products.pkl')

# SPLIT DATASET

In [ ]:
import gc
import pandas as pd
import numpy as np
import os

if __name__ == '__main__':
    path = "/content/data"
    folds = 1

    aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle':'category'})
    departments = pd.read_csv(os.path.join(path, "departments.csv"), dtype={'department_id':np.uint8, 'department': 'category'})
    order_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                                                                      'product_id': np.uint16,
                                                                                      'add_to_cart_order':np.uint8,
                                                                                      'reordered': bool})
    order_train = pd.read_csv(os.path.join(path, "order_products__train.csv"), dtype={'order_id': np.uint32,
                                                                                      'product_id': np.uint16,
                                                                                      'add_to_cart_order':np.uint8,
                                                                                      'reordered': bool})
    orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id':np.uint32,
                                                                  'user_id': np.uint32,
                                                                  'eval_set': 'category',
                                                                  'order_number':np.uint8,
                                                                  'order_dow': np.uint8,
                                                                  'order_hour_of_day': np.uint8
                                                                  })

    products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                                                      'aisle_id': np.uint8,
                                                                      'department_id': np.uint8})

    labels = pd.read_pickle('/content/data/previous_products.pkl')
    orders = orders.loc[(orders.eval_set == 'train') | (orders.eval_set == 'test'), :]
    labels = pd.merge(labels, orders[['order_id', 'user_id', 'eval_set']], on='user_id').drop(['user_id'], axis=1)

    order_train.drop(['add_to_cart_order'], axis=1, inplace=True)

    print('data is loaded')

    orders = np.unique(labels.order_id)

    size = orders.shape[0] // folds

    for fold in range(folds):

        current = orders[fold * size:(fold + 1) * size]

        current = labels.loc[np.in1d(labels.order_id, current), :]

        current = pd.merge(order_train, current, on=['order_id', 'product_id'], how='right')
        current.reordered.fillna(False, inplace=True)
        print(current.columns)
        print(current.shape)

        current.to_pickle('/content/data/chunk_{}.pkl'.format(fold))

data is loaded


<ipython-input-5-8f446552dfb2>:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  current.reordered.fillna(False, inplace=True)


Index(['order_id', 'product_id', 'reordered', 'eval_set'], dtype='object')
(13307953, 4)


<ipython-input-5-8f446552dfb2>:51: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  current.reordered.fillna(False, inplace=True)


# ORDER CONSUM

In [ ]:
import gc
import pandas as pd
import numpy as np
import os
import json
import sklearn.metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from scipy.sparse import dok_matrix, coo_matrix
from sklearn.utils.multiclass import  type_of_target


if __name__ == '__main__':
    path = "/content/data"

    aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle': 'category'})
    departments = pd.read_csv(os.path.join(path, "departments.csv"),
                              dtype={'department_id': np.uint8, 'department': 'category'})
    order_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                                                                      'product_id': np.uint16,
                                                                                      'add_to_cart_order': np.uint8,
                                                                                      'reordered': bool})
    order_train = pd.read_csv(os.path.join(path, "order_products__train.csv"), dtype={'order_id': np.uint32,
                                                                                      'product_id': np.uint16,
                                                                                      'add_to_cart_order': np.uint8,
                                                                                      'reordered': bool})
    orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id': np.uint32,
                                                                  'user_id': np.uint32,
                                                                  'eval_set': 'category',
                                                                  'order_number': np.uint8,
                                                                  'order_dow': np.uint8,
                                                                  'order_hour_of_day': np.uint8
                                                                  })

    products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                                                      'aisle_id': np.uint8,
                                                                      'department_id': np.uint8})

    labels = pd.read_pickle(os.path.join(path, 'chunk_0.pkl'))
    user_product = pd.read_pickle(os.path.join(path, 'previous_products.pkl'))

    order_comsum = orders[['user_id', 'order_number', 'days_since_prior_order']].groupby(['user_id', 'order_number'])\
            ['days_since_prior_order'].sum().groupby(level=[0]).cumsum().reset_index().rename(columns={'days_since_prior_order':'days_since_prior_order_comsum'})

    # order_comsum['days_since_prior_order_comsum'].fillna(0, inplace=True)
    order_comsum.to_pickle('/content/data/orders_comsum.pkl')

    order_comsum = pd.merge(order_comsum, orders, on=['user_id', 'order_number'])[['user_id', 'order_number', 'days_since_prior_order_comsum', 'order_id']]

    order_product = pd.merge(order_prior, orders, on='order_id')[['order_id', 'product_id', 'eval_set']]
    order_product_train_test = labels[['order_id', 'product_id', 'eval_set']]

    order_product = pd.concat([order_product, order_product_train_test])

    order_product = pd.merge(order_product, order_comsum, on='order_id')

    print(order_product.columns)

    order_product = pd.merge(order_product, user_product, on=['user_id', 'product_id'])

    temp = order_product.groupby(['user_id', 'product_id', 'order_number'])['days_since_prior_order_comsum'].sum().groupby(level=[0, 1]).apply(lambda x: np.diff(np.nan_to_num(x)))
    temp = temp.to_frame('periods').reset_index()

    temp.to_pickle('/content/data/product_period.pkl')

    aggregated = temp.copy()
    aggregated['last'] = aggregated.periods.apply(lambda x: x[-1])
    aggregated['prev1'] = aggregated.periods.apply(lambda x: x[-2] if len(x) > 1 else np.nan)
    aggregated['prev2'] = aggregated.periods.apply(lambda x: x[-3] if len(x) > 2 else np.nan)
    aggregated['median'] = aggregated.periods.apply(lambda x: np.median(x[:-1]))
    aggregated['mean'] = aggregated.periods.apply(lambda x: np.mean(x[:-1]))
    aggregated.drop('periods', axis=1, inplace=True)

    aggregated.to_pickle('/content/data/product_periods_stat.pkl')

In [ ]:
import shutil

# Nén folder thành file zip
shutil.make_archive('/content/data', 'zip', '/content/data')

In [ ]:
from google.colab import files

# Tải xuống file zip
files.download('/content/data.zip')

# User product rank

In [ ]:
import gc
import pandas as pd
import numpy as np
import os
import json
import sklearn.metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from scipy.sparse import dok_matrix, coo_matrix
from sklearn.utils.multiclass import  type_of_target


if __name__ == '__main__':
    path = "/content/data"

    aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle': 'category'})
    departments = pd.read_csv(os.path.join(path, "departments.csv"),
                              dtype={'department_id': np.uint8, 'department': 'category'})
    order_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                                                                      'product_id': np.uint16,
                                                                                      'add_to_cart_order': np.uint8,
                                                                                      'reordered': bool})
    order_train = pd.read_csv(os.path.join(path, "order_products__train.csv"), dtype={'order_id': np.uint32,
                                                                                      'product_id': np.uint16,
                                                                                      'add_to_cart_order': np.uint8,
                                                                                      'reordered': bool})
    orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id': np.uint32,
                                                                  'user_id': np.uint32,
                                                                  'eval_set': 'category',
                                                                  'order_number': np.uint8,
                                                                  'order_dow': np.uint8,
                                                                  'order_hour_of_day': np.uint8
                                                                  })

    products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                                                      'aisle_id': np.uint8,
                                                                      'department_id': np.uint8})

    order_train = pd.read_pickle(os.path.join(path, 'chunk_0.pkl'))

    orders_products = pd.merge(orders, order_prior, on="order_id")

    orders_products_products = pd.merge(orders_products, products[['product_id', 'department_id', 'aisle_id']],
                                        on='product_id')

    user_dep_stat = orders_products_products.groupby(['user_id', 'department_id']).agg(
        {'product_id': lambda x: x.nunique(),
         'reordered': 'sum'
         })
    print(user_dep_stat.columns)
    user_dep_stat.rename(columns={'product_id': 'dep_products',
                                  'reordered': 'dep_reordered'}, inplace=True)
    user_dep_stat.reset_index(inplace=True)
    print(user_dep_stat.columns)
    user_dep_stat.to_pickle('/content/data/user_department_products.pkl')

    user_aisle_stat = orders_products_products.groupby(['user_id', 'aisle_id']).agg(
        {'product_id': lambda x: x.nunique(),
         'reordered': 'sum'
         })
    print(user_aisle_stat.columns)
    user_aisle_stat.rename(columns={'product_id': 'aisle_products',
                                    'reordered': 'aisle_reordered'}, inplace=True)
    user_aisle_stat.reset_index(inplace=True)
    user_aisle_stat.to_pickle('/content/data/user_aisle_products.pkl')

Index(['product_id', 'reordered'], dtype='object')
Index(['user_id', 'department_id', 'dep_products', 'dep_reordered'], dtype='object')
Index(['product_id', 'reordered'], dtype='object')


In [ ]:
!zip -r /content/data.zip /content/data/

  adding: content/data/ (stored 0%)
  adding: content/data/departments.csv (deflated 29%)
  adding: content/data/__MACOSX/ (stored 0%)
  adding: content/data/__MACOSX/._order_products__train.csv (deflated 35%)
  adding: content/data/__MACOSX/._aisles.csv (deflated 35%)
  adding: content/data/__MACOSX/._order_products__prior.csv (deflated 35%)
  adding: content/data/__MACOSX/._departments.csv (deflated 35%)
  adding: content/data/__MACOSX/._products.csv (deflated 35%)
  adding: content/data/__MACOSX/._orders.csv (deflated 35%)
  adding: content/data/previous_products.pkl (deflated 70%)
  adding: content/data/orders_comsum.pkl (deflated 88%)
  adding: content/data/product_period.pkl (deflated 86%)
  adding: content/data/user_department_products.pkl (deflated 90%)
  adding: content/data/product_embeddings.pkl (deflated 22%)
  adding: content/data/product_periods_stat.pkl (deflated 85%)
  adding: content/data/chunk_0.pkl (deflated 64%)
  adding: content/data/user_aisle_products.pkl (deflat

In [ ]:
from google.colab import files
files.download('/content/data.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# product2vec dataset


In [ ]:
!unzip /content/drive/MyDrive/data.zip -d /content/data

Archive:  /content/drive/MyDrive/data.zip
replace /content/data/order_products__prior.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
import numpy as np
import os

def create_list(df):
    add_to_cart_order = df.add_to_cart_order.values
    values = df.product_id.values
    index = np.argsort(add_to_cart_order)
    values = values[index].tolist()
    return values



if __name__ == '__main__':
    path = '/content/data'
    order_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                                                                      'product_id': np.uint16,
                                                                                      'add_to_cart_order': np.uint8})
    orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id': np.uint32,
                                                                  'order_dow': np.uint8,
                                                                  'order_hour_of_day': np.uint8
                                                                  })

    data = pd.merge(order_prior, orders, on='order_id')

    data = order_prior.sort_values(['order_id']).groupby('order_id')['product_id'].apply(lambda x: x.tolist()).to_frame('products').reset_index()
    data = pd.merge(data, orders, on='order_id')
    data.to_pickle(os.path.join(path, 'prod2vec.pkl'))

In [ ]:
import pandas as pd
prd2vec = pd.read_pickle('/content/data/prod2vec.pkl')
prd2vec

,order_id,products,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,"[33120, 1819, 43668, 30035, 17794, 40141, 4591...",202279,prior,3,5,9,8.0
1,3,"[46667, 21903, 17704, 17461, 17668, 24838, 337...",205970,prior,16,5,17,12.0
2,4,"[25146, 40285, 17616, 39758, 32645, 22598, 348...",178520,prior,36,1,9,7.0
3,5,"[48825, 27360, 24773, 38693, 47209, 46522, 483...",156122,prior,42,6,16,9.0
4,6,"[15873, 41897, 40462]",22352,prior,4,1,12,30.0
...,...,...,...,...,...,...,...,...
3214869,3421079,[30136],108687,prior,6,1,13,0.0
3214870,3421080,"[25122, 38061, 27845, 31717, 41950, 10667, 493...",52726,prior,2,1,11,2.0
3214871,3421081,"[12861, 35221, 38185, 3060, 20539, 32299, 12218]",117076,prior,1,3,11,NaN
3214872,3421082,"[12738, 47941, 16797, 32700, 17279, 43352, 12023]",175185,prior,23,2,18,4.0


In [ ]:
import warnings
from concurrent.futures import ThreadPoolExecutor

import tensorflow as tf
import numpy as np
import math
from sklearn.utils import shuffle

class Product2VecSkipGram:
    def __init__(self, data, cv_data, batch_size, num_skips, skip_window, vocabulary_size, embedding_size=32,
                 num_negative_sampled=64, len_ratio=0.5):
        self.data = data
        self.cv_data = cv_data
        self.data_index = 0
        self.batch_size = batch_size
        self.num_skips = num_skips
        self.skip_window = skip_window
        self.embedding_size = embedding_size
        self.num_negative_sampled = num_negative_sampled
        self.vocabulary_size = vocabulary_size
        self.len_ratio = len_ratio
        assert batch_size % num_skips == 0
        assert num_skips <= 2 * skip_window
        self.build_model()
        self.loss_list = []

    def predict(self, products):
        results = []
        for i in range(0, len(products), self.batch_size):
            batch = products[i:i+self.batch_size]
            gathered = self.gather(batch)
            results.append(gathered.numpy())
        return np.concatenate(results, axis=0)

    def train(self, num_steps, cv_every_n_steps, cv_steps, lrs):
        optimizer = tf.optimizers.SGD()
        average_loss = 0

        for step in range(num_steps):
            # Adjust learning rate if specified
            if step in lrs:
                optimizer.learning_rate = lrs[step]

            # Generate batch data
            batch_inputs, batch_labels = self.generate_batch()

            # Compute gradients and apply them
            with tf.GradientTape() as tape:
                loss = self.compute_loss(batch_inputs, batch_labels)
            gradients = tape.gradient(loss, self.trainable_variables)
            optimizer.apply_gradients(zip(gradients, self.trainable_variables))

            average_loss += loss.numpy()

            # Print progress every 100 steps
            if step % 1000 == 0:
                if step > 0:
                    average_loss /= 1000
                print(f'Average loss at step {step}: {average_loss}')
                self.loss_list.append(average_loss)
                average_loss = 0

            # Perform cross-validation and save model
            if step % cv_every_n_steps == 0:
                self.data = shuffle(self.data, random_state=0)
                self.save_model(step)
                cv_loss = 0
                for batch_inputs, batch_labels in self.generate_test(cv_steps):
                    loss = self.compute_loss(batch_inputs, batch_labels)
                    cv_loss += loss.numpy()
                print(f'CV loss: {cv_loss / cv_steps}')

    def save_model(self, step):
        self.checkpoint.save(f'./models/prod2vec_skip_gram-{step}')

    def load_model(self, path):
        self.checkpoint.restore(path)

    def build_model(self):
        # Define trainable variables
        self.embeddings = tf.Variable(
            tf.random.uniform([self.vocabulary_size, self.embedding_size], -1.0, 1.0), name="embeddings")
        self.softmax_weights = tf.Variable(
            tf.random.truncated_normal([self.embedding_size, self.vocabulary_size],
                                       stddev=1.0 / math.sqrt(self.embedding_size)), name="softmax_weights")
        self.softmax_biases = tf.Variable(tf.zeros([self.vocabulary_size]), name="softmax_biases")

        # Define checkpoint
        self.checkpoint = tf.train.Checkpoint(embeddings=self.embeddings,
                                              softmax_weights=self.softmax_weights,
                                              softmax_biases=self.softmax_biases)

        # Collect all trainable variables
        self.trainable_variables = [self.embeddings, self.softmax_weights, self.softmax_biases]

    def gather(self, inputs):
        return tf.nn.embedding_lookup(self.embeddings, inputs)

    def compute_loss(self, inputs, labels):
        gathered = self.gather(inputs)
        logits = tf.matmul(gathered, self.softmax_weights) + self.softmax_biases
        return tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))

    def generate_batch(self):
        batch = np.ndarray(shape=(self.batch_size), dtype=np.int32)
        labels = np.ndarray(shape=(self.batch_size), dtype=np.int32)
        counter = 0

        while counter < self.batch_size:
            current = self.data.iloc[self.data_index]
            if len(current) == 1:
                warnings.warn("Length is one", RuntimeWarning)
                self.inc()
                continue

            span = min(2 * self.skip_window + 1, len(current))
            x = target = np.random.randint(0, len(current))
            targets_to_avoid = [x]

            for _ in range(self.num_skips):
                while target in targets_to_avoid and len(targets_to_avoid) != span:
                    target = np.random.randint(0, span)
                if len(targets_to_avoid) == span or counter == self.batch_size:
                    break
                targets_to_avoid.append(target)
                batch[counter] = current[x]
                labels[counter] = current[target]
                counter += 1
            self.inc()

        return batch, labels

    def generate_test(self, num_steps):
        data_index = 0
        for _ in range(num_steps):
            batch = np.ndarray(shape=(self.batch_size), dtype=np.int32)
            labels = np.ndarray(shape=(self.batch_size), dtype=np.int32)
            counter = 0

            while counter < self.batch_size:
                current = self.cv_data.iloc[data_index]
                if len(current) == 1:
                    warnings.warn("Length is one", RuntimeWarning)
                    data_index = self.inc_cv(data_index)
                    continue

                span = min(2 * self.skip_window + 1, len(current))
                x = target = np.random.randint(0, len(current))
                targets_to_avoid = [x]

                for _ in range(self.num_skips):
                    while target in targets_to_avoid and len(targets_to_avoid) != span:
                        target = np.random.randint(0, span)
                    if len(targets_to_avoid) == span or counter == self.batch_size:
                        break
                    targets_to_avoid.append(target)
                    batch[counter] = current[x]
                    labels[counter] = current[target]
                    counter += 1
                data_index = self.inc_cv(data_index)

            yield batch, labels

    def inc(self):
        self.data_index = (self.data_index + 1) % len(self.data)

    def inc_cv(self, data_index):
        return (data_index + 1) % len(self.cv_data)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

if __name__ == '__main__':
    # Set the random seed for reproducibility
    np.random.seed(2017)

    # Load the data
    products = pd.read_csv('/content/data/products.csv')
    df = pd.read_pickle('/content/data/prod2vec.pkl').products
    print('Initial size:', len(df))

    # Split the data into training and validation sets
    df_train, df_cv = train_test_split(df, test_size=0.1, random_state=2017)

    # Hyperparameters
    batch_size = 1024
    rates = {
        100000: 0.5,
        200000: 0.25,
        500000: 0.1,
    }
    num_steps = 120001
    cv_every_n_steps = 20000
    vocabulary_size = np.max(products.product_id) + 1
    embedding_size = 32
    num_skips = 1
    skip_window = 1
    num_negative_sampled = 64

    # Initialize the model
    model = Product2VecSkipGram(
        data=df_train,
        cv_data=df_cv,
        batch_size=batch_size,
        num_skips=num_skips,
        skip_window=skip_window,
        vocabulary_size=vocabulary_size,
        embedding_size=embedding_size,
        num_negative_sampled=num_negative_sampled
    )

    # Train the model
    cv_steps = len(df_cv) // batch_size
    model.train(num_steps=num_steps, cv_every_n_steps=cv_every_n_steps, cv_steps=cv_steps, lrs=rates)


Initial size: 3214874


<ipython-input-8-85a3abf1ac5c>:113: RuntimeWarning: Length is one
  warnings.warn("Length is one", RuntimeWarning)


Average loss at step 0: 10.933735847473145


<ipython-input-8-85a3abf1ac5c>:144: RuntimeWarning: Length is one
  warnings.warn("Length is one", RuntimeWarning)


CV loss: 10.944272035227035
Average loss at step 1000: 10.937376641273499
Average loss at step 2000: 10.925934600830079
Average loss at step 3000: 10.913638600349426
Average loss at step 4000: 10.901537415504455
Average loss at step 5000: 10.889307643890382
Average loss at step 6000: 10.87678489112854
Average loss at step 7000: 10.864795838356018
Average loss at step 8000: 10.853257634162903
Average loss at step 9000: 10.840284879684448
Average loss at step 10000: 10.829845174789428
Average loss at step 11000: 10.815188436508178
Average loss at step 12000: 10.803441907882691
Average loss at step 13000: 10.792400344848632
Average loss at step 14000: 10.780535528182984
Average loss at step 15000: 10.769411598205567
Average loss at step 16000: 10.756149401664734
Average loss at step 17000: 10.74397182559967
Average loss at step 18000: 10.732342823028564
Average loss at step 19000: 10.720555770874023
Average loss at step 20000: 10.707401383399963
CV loss: 10.704675159515283
Average loss at

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

if __name__ == '__main__':
    np.random.seed(2017)

    # Load dữ liệu sản phẩm
    products = pd.read_csv('/content/data/products.csv')
    df = pd.read_pickle('/content/data/prod2vec.pkl').products.tolist()
    print('Initial size:', len(df))

    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    df_train, df_cv = train_test_split(df, test_size=0.1, random_state=2017)

    # Thiết lập các tham số
    batch_size = 128
    rates = {100000: 0.5, 200000: 0.25, 500000: 0.1}

    # Tạo mô hình Product2VecSkipGram
    model = Product2VecSkipGram(
        data=df_train,
        cv_data=df_cv,
        batch_size=batch_size,
        num_skips=1,
        skip_window=1,
        vocabulary_size=np.max(products.product_id) + 1
    )

    # Tải trọng số mô hình đã lưu
    checkpoint_path = '/content/models/prod2vec_skip_gram-120000-7'
    model.load_model(checkpoint_path)

    # Dự đoán embeddings cho các sản phẩm
    product_ids = products.product_id.values
    embeddings = model.predict(product_ids)

    # Kết hợp embeddings với dữ liệu sản phẩm
    embeddings_df = pd.DataFrame(embeddings)
    products = pd.concat([products, embeddings_df], axis=1)

    # Lưu embeddings vào file pickle
    products.to_pickle('/content/data/product_embeddings_final.pkl')


Initial size: 3214874


In [ ]:
import pandas as pd
product_embedđings = pd.read_pickle('/content/data/product_embeddings_final.pkl')
product_embedđings

,product_id,product_name,aisle_id,department_id,0,1,2,3,4,5,...,22,23,24,25,26,27,28,29,30,31
0,1,Chocolate Sandwich Cookies,61,19,0.577945,-0.096393,0.003814,-0.797471,-1.205676,0.216385,...,-0.417261,-0.438668,0.916984,0.205373,0.732128,-0.579414,0.897574,-0.123179,0.558207,0.650109
1,2,All-Seasons Salt,104,13,-0.368773,-0.925426,-0.258932,-0.137203,-0.239864,0.764848,...,-0.621131,-0.901391,0.926375,-0.412561,-0.217289,-0.080136,0.881149,0.387462,-0.405105,0.173570
2,3,Robust Golden Unsweetened Oolong Tea,94,7,0.440653,-0.619670,0.179074,-0.445268,0.410423,-0.459891,...,-0.912754,0.864426,-0.543560,-0.609864,-0.332112,-0.379196,0.382682,0.275744,-0.284038,0.112095
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,0.128498,0.723323,-0.201369,0.773948,-0.156643,0.744697,...,0.644185,0.692339,-0.538094,0.388297,-0.474511,0.758511,-0.705910,-0.484293,-0.305427,0.974226
4,5,Green Chile Anytime Sauce,5,13,-0.749400,-0.647876,-0.100347,0.329370,-0.063627,0.038270,...,0.839368,0.819475,-0.729061,0.874731,-0.629781,-0.759848,-0.373825,0.937420,0.432422,-0.999377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5,-0.467433,0.274614,-0.247119,0.242598,-0.746269,0.735775,...,0.943806,0.698218,0.650805,-0.196231,-0.489749,0.265844,-0.813873,0.518717,0.894229,0.623529
49684,49685,En Croute Roast Hazelnut Cranberry,42,1,0.404030,0.015798,0.260449,-0.256866,0.604304,-0.902415,...,-0.499875,-0.908838,-0.547454,0.405935,0.186130,-0.654073,0.255263,-0.663399,-0.912833,0.746811
49685,49686,Artisan Baguette,112,3,0.376969,0.764537,0.148883,-0.570026,-0.378887,-0.994709,...,-0.867331,0.175235,-0.081328,0.936420,-0.820358,-0.624066,-0.151258,-0.515862,0.762789,-0.172248
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8,0.591801,0.473472,0.572136,0.904277,-0.612168,-0.070111,...,-0.443655,-0.789429,-0.923802,-0.091967,0.492270,-0.073420,0.795772,0.321352,0.452993,0.225522


In [ ]:
!zip -r /content/models.zip /content/models

  adding: content/models/ (stored 0%)
  adding: content/models/prod2vec_skip_gram-0-1.index (deflated 37%)
  adding: content/models/prod2vec_skip_gram-20000-2.index (deflated 37%)
  adding: content/models/prod2vec_skip_gram-60000-4.data-00000-of-00001 (deflated 8%)
  adding: content/models/prod2vec_skip_gram-80000-5.index (deflated 37%)
  adding: content/models/prod2vec_skip_gram-120000-7.index (deflated 37%)
  adding: content/models/prod2vec_skip_gram-0-1.data-00000-of-00001 (deflated 9%)
  adding: content/models/prod2vec_skip_gram-100000-6.data-00000-of-00001 (deflated 8%)
  adding: content/models/prod2vec_skip_gram-40000-3.index (deflated 37%)
  adding: content/models/checkpoint (deflated 42%)
  adding: content/models/prod2vec_skip_gram-20000-2.data-00000-of-00001 (deflated 8%)
  adding: content/models/prod2vec_skip_gram-120000-7.data-00000-of-00001 (deflated 8%)
  adding: content/models/prod2vec_skip_gram-80000-5.data-00000-of-00001 (deflated 8%)
  adding: content/models/prod2vec_s

In [ ]:
from google.colab import files
files.download('/content/models.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/data.zip /content/data

  adding: content/data/ (stored 0%)
  adding: content/data/product_periods_stat.pkl (deflated 85%)
  adding: content/data/.ipynb_checkpoints/ (stored 0%)
  adding: content/data/orders.csv (deflated 70%)
  adding: content/data/order_products__train.csv (deflated 71%)
  adding: content/data/prod2vec.pkl (deflated 55%)
  adding: content/data/departments.csv (deflated 29%)
  adding: content/data/order_products__prior.csv (deflated 72%)
  adding: content/data/chunk_0.pkl (deflated 64%)
  adding: content/data/products.csv (deflated 63%)
  adding: content/data/__MACOSX/ (stored 0%)
  adding: content/data/__MACOSX/._products.csv (deflated 35%)
  adding: content/data/__MACOSX/._orders.csv (deflated 35%)
  adding: content/data/__MACOSX/._aisles.csv (deflated 35%)
  adding: content/data/__MACOSX/._order_products__prior.csv (deflated 35%)
  adding: content/data/__MACOSX/._departments.csv (deflated 35%)
  adding: content/data/__MACOSX/._order_products__train.csv (deflated 35%)
  adding: content/dat

In [ ]:
from google.colab import files
files.download('/content/data.zip')

In [ ]:
!pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 34.8 MB/s eta 0:00:00


In [ ]:
import gc
import pandas as pd
import numpy as np
import os
import lightgbm as lgb
import json
import sklearn.metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from scipy.sparse import dok_matrix, coo_matrix
from sklearn.utils.multiclass import  type_of_target



def fscore(true_value_matrix, prediction, order_index, product_index, rows, cols, threshold=[0.5]):

    prediction_value_matrix = coo_matrix((prediction, (order_index, product_index)), shape=(rows, cols), dtype=np.float32)
    # prediction_value_matrix.eliminate_zeros()

    return list(map(lambda x: f1_score(true_value_matrix, prediction_value_matrix > x, average='samples'), threshold))


if __name__ == '__main__':
    path = "/content/data"

    aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle': 'category'})
    departments = pd.read_csv(os.path.join(path, "departments.csv"),
                              dtype={'department_id': np.uint8, 'department': 'category'})
    order_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                                                                      'product_id': np.uint16,
                                                                                      'add_to_cart_order': np.uint8,
                                                                                      'reordered': bool})
    order_train = pd.read_csv(os.path.join(path, "order_products__train.csv"), dtype={'order_id': np.uint32,
                                                                                      'product_id': np.uint16,
                                                                                      'add_to_cart_order': np.uint8,
                                                                                      'reordered': bool})
    orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id': np.uint32,
                                                                  'user_id': np.uint32,
                                                                  'eval_set': 'category',
                                                                  'order_number': np.uint8,
                                                                  'order_dow': np.uint8,
                                                                  'order_hour_of_day': np.uint8
                                                                  })

    products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                                                      'aisle_id': np.uint8,
                                                                      'department_id': np.uint8})

    product_embeddings = pd.read_pickle('/content/data/product_embeddings_final.pkl')
    embedings = list(range(32))
    product_embeddings = product_embeddings[embedings + ['product_id']]

    order_train = pd.read_pickle(os.path.join(path, 'chunk_0.pkl'))
    order_test = order_train.loc[order_train.eval_set == "test", ['order_id', 'product_id']]
    order_train = order_train.loc[order_train.eval_set == "train", ['order_id',  'product_id',  'reordered']]

    product_periods = pd.read_pickle(os.path.join(path, 'product_periods_stat.pkl')).fillna(9999)

    print(order_train.columns)

    ###########################

    prob = pd.merge(order_prior, orders, on='order_id')
    print(prob.columns)
    prob = prob.groupby(['product_id', 'user_id'])\
        .agg({'reordered':'sum', 'user_id': 'size'})
    print(prob.columns)

    prob.rename(columns={'sum': 'reordered',
                         'user_id': 'total'}, inplace=True)

    prob.reordered = (prob.reordered > 0).astype(np.float32)
    prob.total = (prob.total > 0).astype(np.float32)
    prob['reorder_prob'] = prob.reordered / prob.total
    prob = prob.groupby('product_id').agg({'reorder_prob': 'mean'}).rename(columns={'mean': 'reorder_prob'})\
        .reset_index()


    prod_stat = order_prior.groupby('product_id').agg({'reordered': ['sum', 'size'],
                                                       'add_to_cart_order':'mean'})
    prod_stat.columns = prod_stat.columns.levels[1]
    prod_stat.rename(columns={'sum':'prod_reorders',
                              'size':'prod_orders',
                              'mean': 'prod_add_to_card_mean'}, inplace=True)
    prod_stat.reset_index(inplace=True)

    prod_stat['reorder_ration'] = prod_stat['prod_reorders'] / prod_stat['prod_orders']

    prod_stat = pd.merge(prod_stat, prob, on='product_id')

    # prod_stat.drop(['prod_reorders'], axis=1, inplace=True)

    user_stat = orders.loc[orders.eval_set == 'prior', :].groupby('user_id').agg({'order_number': 'max',
                                                                                  'days_since_prior_order': ['sum',
                                                                                                             'mean',
                                                                                                             'median']})
    user_stat.columns = user_stat.columns.droplevel(0)
    user_stat.rename(columns={'max': 'user_orders',
                              'sum': 'user_order_starts_at',
                              'mean': 'user_mean_days_since_prior',
                              'median': 'user_median_days_since_prior'}, inplace=True)
    user_stat.reset_index(inplace=True)

    orders_products = pd.merge(orders, order_prior, on="order_id")

    user_order_stat = orders_products.groupby('user_id').agg({'user_id': 'size',
                                                              'reordered': 'sum',
                                                              "product_id": lambda x: x.nunique()})

    user_order_stat.rename(columns={'user_id': 'user_total_products',
                                    'product_id': 'user_distinct_products',
                                    'reordered': 'user_reorder_ratio'}, inplace=True)

    user_order_stat.reset_index(inplace=True)
    user_order_stat.user_reorder_ratio = user_order_stat.user_reorder_ratio / user_order_stat.user_total_products

    user_stat = pd.merge(user_stat, user_order_stat, on='user_id')
    user_stat['user_average_basket'] = user_stat.user_total_products / user_stat.user_orders

    ########################### products

    prod_usr = orders_products.groupby(['product_id']).agg({'user_id': lambda x: x.nunique()})
    prod_usr.rename(columns={'user_id':'prod_users_unq'}, inplace=True)
    prod_usr.reset_index(inplace=True)

    prod_usr_reordered = orders_products.loc[orders_products.reordered, :].groupby(['product_id']).agg({'user_id': lambda x: x.nunique()})
    prod_usr_reordered.rename(columns={'user_id': 'prod_users_unq_reordered'}, inplace=True)
    prod_usr_reordered.reset_index(inplace=True)

    order_stat = orders_products.groupby('order_id').agg({'order_id': 'size'}) \
        .rename(columns={'order_id': 'order_size'}).reset_index()

    orders_products = pd.merge(orders_products, order_stat, on='order_id')
    orders_products['add_to_cart_order_inverted'] = orders_products.order_size - orders_products.add_to_cart_order
    orders_products['add_to_cart_order_relative'] = orders_products.add_to_cart_order / orders_products.order_size

    data = orders_products.groupby(['user_id', 'product_id']).agg({'user_id': 'size',
                                                                   'order_number': ['min', 'max'],
                                                                   'add_to_cart_order': ['mean', 'median'],
                                                                   'days_since_prior_order': ['mean', 'median'],
                                                                   'order_dow': ['mean', 'median'],
                                                                   'order_hour_of_day': ['mean', 'median'],
                                                                   'add_to_cart_order_inverted': ['mean', 'median'],
                                                                   'add_to_cart_order_relative': ['mean', 'median'],
                                                                   'reordered': ['sum']})

    data.columns = data.columns.droplevel(0)
    data.columns = ['up_orders', 'up_first_order', 'up_last_order', 'up_mean_cart_position', 'up_median_cart_position',
                    'days_since_prior_order_mean', 'days_since_prior_order_median', 'order_dow_mean',
                    'order_dow_median',
                    'order_hour_of_day_mean', 'order_hour_of_day_median',
                    'add_to_cart_order_inverted_mean', 'add_to_cart_order_inverted_median',
                    'add_to_cart_order_relative_mean', 'add_to_cart_order_relative_median',
                    'reordered_sum'
                    ]

    data['user_product_reordered_ratio'] = (data.reordered_sum + 1.0) / data.up_orders

    # data['first_order'] = data['up_orders'] > 0
    # data['second_order'] = data['up_orders'] > 1
    #
    # data.groupby('product_id')['']

    data.reset_index(inplace=True)

    data = pd.merge(data, prod_stat, on='product_id')
    data = pd.merge(data, user_stat, on='user_id')

    data['up_order_rate'] = data.up_orders / data.user_orders
    data['up_orders_since_last_order'] = data.user_orders - data.up_last_order
    data['up_order_rate_since_first_order'] = data.user_orders / (data.user_orders - data.up_first_order + 1)

    ############################

    user_dep_stat = pd.read_pickle('/content/data/user_department_products.pkl')
    user_aisle_stat = pd.read_pickle('/content/data/user_aisle_products.pkl')

    ############### train

    print(order_train.shape)
    order_train = pd.merge(order_train, products, on='product_id')
    print(order_train.shape)
    order_train = pd.merge(order_train, orders, on='order_id')
    print(order_train.shape)
    order_train = pd.merge(order_train, user_dep_stat, on=['user_id', 'department_id'])
    print(order_train.shape)
    order_train = pd.merge(order_train, user_aisle_stat, on=['user_id', 'aisle_id'])
    print(order_train.shape)

    order_train = pd.merge(order_train, prod_usr, on='product_id')
    print(order_train.shape)
    order_train = pd.merge(order_train, prod_usr_reordered, on='product_id', how='left')
    order_train.prod_users_unq_reordered.fillna(0, inplace=True)
    print(order_train.shape)

    order_train = pd.merge(order_train, data, on=['product_id', 'user_id'])
    print(order_train.shape)

    order_train['aisle_reordered_ratio'] = order_train.aisle_reordered / order_train.user_orders
    order_train['dep_reordered_ratio'] = order_train.dep_reordered / order_train.user_orders

    order_train = pd.merge(order_train, product_periods, on=['user_id',  'product_id'])

    ##############

    order_test = pd.merge(order_test, products, on='product_id')
    order_test = pd.merge(order_test, orders, on='order_id')
    order_test = pd.merge(order_test, user_dep_stat, on=['user_id', 'department_id'])
    order_test = pd.merge(order_test, user_aisle_stat, on=['user_id', 'aisle_id'])

    order_test = pd.merge(order_test, prod_usr, on='product_id')
    order_test = pd.merge(order_test, prod_usr_reordered, on='product_id', how='left')
    order_train.prod_users_unq_reordered.fillna(0, inplace=True)

    order_test = pd.merge(order_test, data, on=['product_id', 'user_id'])

    order_test['aisle_reordered_ratio'] = order_test.aisle_reordered / order_test.user_orders
    order_test['dep_reordered_ratio'] = order_test.dep_reordered / order_test.user_orders

    order_test = pd.merge(order_test, product_periods, on=['user_id', 'product_id'])

    order_train = pd.merge(order_train, product_embeddings, on=['product_id'])
    order_test = pd.merge(order_test, product_embeddings, on=['product_id'])

    print('data is joined')

Index(['order_id', 'product_id', 'reordered'], dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered', 'user_id',
       'eval_set', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order'],
      dtype='object')
Index(['reordered', 'user_id'], dtype='object')
(8474661, 3)
(8474661, 6)
(8474661, 12)
(8474661, 14)
(8474661, 16)
(8474661, 17)


<ipython-input-14-86b87bf047f2>:193: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  order_train.prod_users_unq_reordered.fillna(0, inplace=True)


(8474661, 18)
(8474661, 51)


<ipython-input-14-86b87bf047f2>:213: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  order_train.prod_users_unq_reordered.fillna(0, inplace=True)


data is joined
not included {'order_hour_of_day_median', 'user_median_days_since_prior', 'up_median_cart_position', 'user_id', 'order_id', 'eval_set', 'reordered', 'add_to_cart_order_inverted_median', 'add_to_cart_order_relative_median', 'order_dow_median', 'days_since_prior_order_median', 'product_name'}


In [ ]:
order_train.to_pickle('/content/drive/MyDrive/Data_rec_sys/order_train.pkl')
order_test.to_pickle('/content/drive/MyDrive/Data_rec_sys/order_test.pkl')
print("Saved successfully")

Saved successfully


In [ ]:
import pandas as pd
order_train = pd.read_pickle('/content/drive/MyDrive/Data_rec_sys/order_train.pkl')
order_test = pd.read_pickle('/content/drive/MyDrive/Data_rec_sys/order_test.pkl')

In [ ]:
features = [
    # 'reordered_dow_ration', 'reordered_dow', 'reordered_dow_size',
    # 'reordered_prev', 'add_to_cart_order_prev', 'order_dow_prev', 'order_hour_of_day_prev',
    'user_product_reordered_ratio', 'reordered_sum',
    'add_to_cart_order_inverted_mean', 'add_to_cart_order_relative_mean',
    'reorder_prob',
    'last', 'prev1', 'prev2', 'median', 'mean',
    'dep_reordered_ratio', 'aisle_reordered_ratio',
    'aisle_products',
    'aisle_reordered',
    'dep_products',
    'dep_reordered',
    'prod_users_unq', 'prod_users_unq_reordered',
    'order_number', 'prod_add_to_card_mean',
    'days_since_prior_order',
    'order_dow', 'order_hour_of_day',
    'reorder_ration',
    'user_orders', 'user_order_starts_at', 'user_mean_days_since_prior',
    # 'user_median_days_since_prior',
    'user_average_basket', 'user_distinct_products', 'user_reorder_ratio', 'user_total_products',
    'prod_orders', 'prod_reorders',
    'up_order_rate', 'up_orders_since_last_order', 'up_order_rate_since_first_order',
    'up_orders', 'up_first_order', 'up_last_order', 'up_mean_cart_position',
    # 'up_median_cart_position',
    'days_since_prior_order_mean',
    # 'days_since_prior_order_median',
    'order_dow_mean',
    # 'order_dow_median',
    'order_hour_of_day_mean',
    # 'order_hour_of_day_median'
]
features.extend(embedings)
categories = ['product_id', 'aisle_id', 'department_id']
cat_features = ','.join(map(lambda x: str(x + len(features)), range(len(categories))))
features.extend(categories)

print('not included', set(order_train.columns.tolist()) - set(features))

data = order_train[features]
labels = order_train[['reordered']].values.astype(np.float32).flatten()

data_val = order_test[features]

assert data.shape[0] == 8474661

,user_product_reordered_ratio,reordered_sum,add_to_cart_order_inverted_mean,add_to_cart_order_relative_mean,reorder_prob,last,prev1,prev2,median,mean,...,25,26,27,28,29,30,31,product_id,aisle_id,department_id
0,1.0,4,8.600000,0.191919,0.554809,30.0,54.0,30.0,34.5,37.75,...,0.084274,-0.696017,0.148283,-0.459727,0.105391,-1.003278,-0.133003,33120,86,16
1,1.0,4,7.400000,0.304747,0.542653,84.0,30.0,39.0,25.0,24.25,...,-0.128308,-0.708541,0.014055,0.217084,-0.075128,-1.389179,0.048452,28985,83,4
2,1.0,0,6.000000,0.333333,0.120098,153.0,9999.0,9999.0,9999.0,9999.00,...,-0.234860,-0.560458,0.489395,-0.731145,-0.148209,0.168253,0.886372,9327,104,13
3,1.0,4,7.200000,0.424183,0.188540,60.0,24.0,30.0,27.0,25.25,...,0.735302,-0.705822,0.513517,0.815715,0.799510,0.594794,0.347346,45918,19,13
4,1.0,2,7.666667,0.415132,0.367857,60.0,54.0,39.0,46.5,46.50,...,0.305714,-0.224656,0.933523,0.062477,0.016741,-0.825738,0.836887,30035,17,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8474656,1.0,0,2.000000,0.714286,0.272727,187.0,9999.0,9999.0,9999.0,9999.00,...,0.078482,0.600951,0.445606,0.498717,0.724301,0.293876,0.811392,20539,69,15
8474657,1.0,0,1.000000,0.857143,0.581696,187.0,9999.0,9999.0,9999.0,9999.00,...,-0.510854,0.515423,-0.696948,-0.337312,0.813884,-0.313859,-0.199995,35221,115,7
8474658,1.0,0,0.000000,1.000000,0.230769,187.0,9999.0,9999.0,9999.0,9999.00,...,-0.007271,-0.695404,0.718614,-0.344594,-0.471947,0.404656,0.871768,12861,67,20
8474659,1.0,0,1.000000,0.857143,0.394916,68.0,9999.0,9999.0,9999.0,9999.00,...,0.627502,-0.921000,0.781783,0.332681,0.043629,0.710960,0.380609,12023,117,19


In [ ]:
print(cat_features)
print(data.columns)

107,108,109
Index([   'user_product_reordered_ratio',                   'reordered_sum',
       'add_to_cart_order_inverted_mean', 'add_to_cart_order_relative_mean',
                          'reorder_prob',                            'last',
                                 'prev1',                           'prev2',
                                'median',                            'mean',
       ...
                                      25,                                26,
                                      27,                                28,
                                      29,                                30,
                                      31,                      'product_id',
                              'aisle_id',                   'department_id'],
      dtype='object', length=110)
